# Отток клиентов

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

## 1.1. Проверка качества данных 

In [ ]:
#импортируем библиотеки необходимые для подготовки данных
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv')
data.head()

целевой признак Exited — факт ухода клиента — категориальный признак => задача будет достигаться методами классификации.

In [ ]:
data.info()

In [ ]:
data['Tenure'] = data['Tenure'].fillna(-1)

In [ ]:
data['Geography'].unique()

Датасет содержить 10000 строк, явных пропусков не обнаружено.  
Требуемые изменения: 
1. RowNumber — столбец дублирует информацию стандартного датафрейма и не несетя ценности для обучения модели. Поэтому столбец можно удалить
2. CustomerId — является идентификатором пользователя (по данному столбцу значению необходимо проверить наличие дубликатов) и если отсутсвуют — стоблец также можно удалить. 	
3. Surname — является идентифицирующей пользователя информацией, содержит 2932 уникальных значений (данные kaggle), что сильно усложняет задачу, но не несет ценности для качества будущей модели — столбец можно удалить. 
4. CreditScore — следует проверить распределение и сохранить данный столбец. 
5. Geography — всего 3 уникальных значения France', 'Spain', 'Germany' — выполнить преобразование OHE / ordinal encoding 
6. Gender — значение преобразовать — OHE / ordinal encoding. 
7. Age — Exited — тип данных верны, требуется проработка OHE. 

In [ ]:
data.duplicated().sum()

In [ ]:
data['CustomerId'].value_counts().sum()

Дупликатов в данных не обнаружено.  
Приступим к изменению оптимизации всего датафрейма. Сначала уберем лишние столбцы. 

## 1.2. Подготовка данных для машинного обучения

In [ ]:
#Обозначим новый датафрейм "data_ml" — данные для машинного обучения
#Удалим столбцы-идентификаторы, не представляющие ценностия для алгоритма 
#for_drop = ['RowNumber','CustomerId', 'Surname']
for_drop = ['RowNumber','CustomerId', 'Surname']
data_ml = data.drop(for_drop, axis=1)
data_ml.head()

In [ ]:
data_ml.shape

Данные подготовим методом OHE, что позволит нам использовать модели, деревое решений, случайный лес и логистическую регрессию.

In [ ]:
data_ml = pd.get_dummies(data_ml, drop_first=True)
data_ml.head()

In [ ]:
data_ml.shape

В новом датафрейме мы получили 12 столбцов.  
Столбец Geography содержал 3 класса,  Gender_Male 2 класса — в результате мы получили только 2 и 1 столбцов соответственно, что позволяет избежать дами-ловушки. 

## 1.3. Формирование выборок: обучающая, валидационная, тестовая

В нашем случае нет отдельно выделенной тестовой выборки, поэтому существющие данные мы делим на три выборки: 
- обучающая 60% для обучения модели
- валидационная 20% для выбора лучшего алгоритма и оптимальных параметров
- тестовая 20% для финального теста лучшей модели.

In [ ]:
#разделим на признаки и целевой признак
features = data_ml.drop('Exited', axis=1)
target = data_ml['Exited']

In [ ]:
features.head()

In [ ]:
target.head()

#### Разделение выборки произведем в 2 этапа:
1. Выделим валидационную 60%
2. Разделим оставшиеся 40% на 2 равные части (валидационная и тестовая)

In [ ]:
#1. Выделим валидационную 60%
features_train, features_validtest, target_train, target_validtest = train_test_split(features,
                                                    target,
                                                    train_size=0.6,
                                                    random_state=12345, 
                                                    stratify=target)

In [ ]:
print('Признаки обучающей выборки:',features_train.shape,  
      'Целевой признак обучающей выборки:', target_train.shape, 
      'Валидационная и тестовая вместе', features_validtest.shape, target_validtest.shape)

In [ ]:
#2. Разделим оставшиеся 40% на 2 равные части (валидационная и тестовая)
features_valid, features_test, target_valid, target_test = train_test_split(features_validtest,
                                                    target_validtest,
                                                    train_size=0.5,
                                                    random_state=12345, 
                                                    stratify=target_validtest)

In [ ]:
print(features_valid.shape, target_valid.shape, features_test.shape, target_test.shape)

Полученые выбрки:   
**Обучающая**   
features_train  
target_train  

**Валидационная**   
features_valid  
target_valid  

**Тестовая**   
features_test  
target_test  

## 1.4. Масштабирование признаков

In [ ]:
#импортируем библиотеку для стандартноо масштабирования
from sklearn.preprocessing import StandardScaler

In [ ]:
features_train.head()

In [ ]:
#Для масштабирования зафиксируем численные признаки
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [ ]:
#Создадим объект этой структуры и настроим его на обучающих данных:
scaler = StandardScaler()
scaler.fit(features_train[numeric])


In [ ]:
#Масштабируем численные признаки обучающей выборки 
features_train[numeric] = scaler.transform(features_train[numeric])
features_train.head()

In [ ]:
#Масштабируем численные признаки валидационной выборки 
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_valid.head()

In [ ]:
#Масштабируем численные признаки тестовой выборки 
features_test[numeric] = scaler.transform(features_test[numeric])
features_test.head()

В результате мы получили масштабированные признаки для трех выборок. Перейдем к исследованию задачим моделями машинного обучения. 

# 2. Исследование задачи

Для исследовательского обучения используем три модели

2.1 Обучение моделей

In [ ]:
#импорт библиотек трех моделей
#Деревое решений
from sklearn.tree import DecisionTreeClassifier

#Случайный лес 
from sklearn.ensemble import RandomForestClassifier

#Логистическая регрессия
from sklearn.linear_model import LogisticRegression

In [ ]:
#Импортируем необходимые метики
#метика accuracy
from sklearn.metrics import accuracy_score

#матрица ошибок
from sklearn.metrics import confusion_matrix

#полнота
from sklearn.metrics import recall_score

#точность
from sklearn.metrics import precision_score

#F-1 мера
from sklearn.metrics import f1_score

#AUC-ROC
from sklearn.metrics import roc_auc_score

#ROC-кривая
from sklearn.metrics import roc_curve

**2.1. Точность разных моделей**

In [ ]:
def all_models_accuracy(features_train, target_train, features_valid, target_valid):
    model_DTC = DecisionTreeClassifier(random_state=123)
    DTC_score = model_DTC.fit(features_train, target_train).score(features_valid, target_valid)
    
    model_RFC = RandomForestClassifier(random_state=12345, n_estimators = 100)
    RFC_score = model_RFC.fit(features_train, target_train).score(features_valid, target_valid)
    
    model_LgR = LogisticRegression(solver = 'liblinear')
    LgR_score = model_LgR.fit(features_train, target_train).score(features_valid, target_valid)
    print("Точность:" "дерево решений", DTC_score, "случайный лес ", RFC_score, "логистческая регрессия", LgR_score)

In [ ]:
all_models_accuracy(features_train, target_train, features_valid, target_valid)

**2.2 Проверка баланса классов выборки**

In [ ]:
target_train.value_counts(normalize = 1)

In [ ]:
target_valid.value_counts(normalize = 1)

У обучающией и валидационной выборок наблюдается заметный дисбаланс класов. Ответы 0 ≈80% , 1 ≈ 20%  
Можно ожидать, что результаты предсказаний моделей будут сильно склоняться к одному варианту ответов. 

**2.3 Проверка адекватности**

Проверим какое соотношение ответов выдает каждая модель.

In [ ]:
def all_models_share(features_train, target_train, features_valid, target_valid):
    model_DTC = DecisionTreeClassifier(random_state=123)
    model_DTC.fit(features_train, target_train)
    DTC_share = pd.Series(model_DTC.predict(features_valid)).value_counts(normalize = 1)
    
    
    
    model_RFC = RandomForestClassifier(random_state=12345, n_estimators = 100)
    model_RFC.fit(features_train, target_train)
    RFC_share = pd.Series(model_RFC.predict(features_valid)).value_counts(normalize = 1)
    
    model_LgR = LogisticRegression(solver = 'liblinear')
    model_LgR.fit(features_train, target_train)
    LgR_share = pd.Series(model_LgR.predict(features_valid)).value_counts(normalize = 1)
    

    
    print("Доли ответов:" "дерево решений", DTC_share, "случайный лес ", RFC_share, "логистческая регрессия", LgR_share , end='')

In [ ]:
all_models_share(features_train, target_train, features_valid, target_valid)

Логистичесая регрессия показывала самую высокую точность, но данная модель выдает 100% одинаковый результат = 0  
Сравним качество предсказаний с константной моделью. За константу примем пример 0, так как это значение встречается 80% случаев 

In [ ]:
#Создаем константную модель
target_predict_constant = pd.Series([0]*len(target_valid))
target_predict_constant.shape

In [ ]:
accuracy_score_constant = accuracy_score(target_valid, target_predict_constant)
accuracy_score_constant

Точность моделей: дерево решений 0.2915 случайный лес  0.5 логистческая регрессия 0.796  
Точность константной модели: 0.796    
Ни одна модель не показал результат выше констентной модели, вероятно проблема кроется в дисбалансе классов.    

Чтобы разобраться подбробнее ознакомися с матрицой ошибок для каждой модели.  

In [ ]:
#матрица ошибок для дерево решений
model_DTC = DecisionTreeClassifier(random_state=123)
model_DTC.fit(features_train, target_train)
DTC_prediction = model_DTC.predict(features_valid)
confusion_matrix(target_valid, DTC_prediction)


Дерево решений склонно выдавать позитивные предсказания, очень высокое количество ложных позитивных предсказания (FP)  
Изучим полноту, точность и F1-меру

In [ ]:
def rec_prec_f1(target_valid, prediction):
    print("Полнота" , recall_score(target_valid, prediction))
    print("Точность", precision_score(target_valid, prediction))
    print("F1-мера", f1_score(target_valid, prediction))
    print("AUC-ROC", roc_auc_score(target_valid, prediction))
    

In [ ]:
#полнота, точность и F1-мера для дерева решений
rec_prec_f1(target_valid, DTC_prediction)

Низкое значние F1- меры свидетельствует о низком качестве модели, проблема в точности. 

In [ ]:
#матрица ошибок для случайного леса
model_RFC = RandomForestClassifier(random_state=12345, n_estimators = 100)
model_RFC.fit(features_train, target_train)
RFC_prediction = model_RFC.predict(features_valid)
confusion_matrix(target_valid, RFC_prediction)


Случайный лес примерно равную пропорцию позитивных и негативных предсказаний, выдает заметно больше качественных предсказаний, но также сильно склоняется к ложно позитивным предсказаниям (FP)

In [ ]:
#полнота, точность и F1-мера для случайного леса
rec_prec_f1(target_valid, RFC_prediction)

Низкое значние F1- меры свидетельствует о низком качестве модели, проблема в точности. 

In [ ]:
#матрица ошибок для логистической регрессии
model_LgR = LogisticRegression(solver = 'liblinear')
model_LgR.fit(features_train, target_train)
LgR_prediction = model_LgR.predict(features_valid)
confusion_matrix(target_valid, LgR_prediction)



Для логистической регрессии расчитаем метрику AUC-ROC

In [ ]:
LgR_probabilities_one_valid = model_LgR.predict_proba(features_valid)[:, 1]

auc_roc_LgR = roc_auc_score(target_valid, LgR_probabilities_one_valid)

auc_roc_LgR

In [ ]:
fpr, tpr, thresholds = roc_curve(target_valid, LgR_probabilities_one_valid) 

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

In [ ]:
LgR_probabilities_one_valid = model_LgR.predict_proba(features_valid)[:, 1]

auc_roc_LgR = roc_auc_score(target_valid, LgR_probabilities_one_valid)

auc_roc_LgR

In [ ]:
fpr, tpr, thresholds = roc_curve(target_valid, LgR_probabilities_one_valid) 

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

In [ ]:
#попробуем обучать логистическую регресию сбалансировав классы
model_LgR = LogisticRegression(solver = 'liblinear', class_weight='balanced')
model_LgR.fit(features_train, target_train)
LgR_probabilities_one_valid_class_weight = model_LgR.predict_proba(features_valid)[:, 1]
print("Score", model_LgR.score(features_valid, target_valid))
print("AUC-ROC", roc_auc_score(target_valid, LgR_probabilities_one_valid_class_weight))

fpr, tpr, thresholds = roc_curve(target_valid, LgR_probabilities_one_valid_class_weight) 
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()


Точность предсказаний не улучшилась.

Логистическая регрессия выдает 100% негативных предсказаний — высокая точность модели объясняется высокой долей негативных ответов в валидационной выборке.  

Перейдем к формированию сбалансированных выборок

# 3. Исследование и обучение моделей на сбалансированной выборке

## 3.1 Борьба с дисбалансом

In [ ]:
from sklearn.utils import shuffle

Как мы выяснили ранее в нашей выборке отрицательны ответов ≈80% , положитительных ≈ 20%.   
Нам необходмо увеличить количество положительных ответов в 4 раза для достижения баланса.   

1. Разделим обучающую выборку на отрицательные и положительные объекты;
2. Скопируем несколько раз положительные объекты;
3. С учётом полученных данных создадим новую обучающую выборку;
4. Перемешаем данные: идущие друг за другом одинаковые вопросы не помогут обучению.
5. Обучим новые модели

In [ ]:
#Ознакомимся с пероначальным распределением классов
target_train.value_counts(normalize = 1)

In [ ]:
target_train.shape

In [ ]:
target_train.plot(kind ='hist', bins=2, figsize=(1,1))

In [ ]:
#создадим функцию для увеличения представленной класса в выборке 
def upsample(features, target, repeat, upsampled_сlass):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    if upsampled_сlass == 0:
        features_upsampled = pd.concat([features_zeros]* repeat + [features_ones] )
        target_upsampled = pd.concat([target_zeros]* repeat + [target_ones] )
        features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
        
    elif upsampled_сlass == 1:
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
        features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    else:
        features_upsampled = 0
        target_upsampled = 0  
        
        
       
    return features_upsampled, target_upsampled
    "Функция принимаем значение признаков (features[]), целевого признака (target[]), repeat(int / float), "
    " класс который будет увеличен (upsampled_сlass (0 or 1))"

In [ ]:
#Протестируем функцию (верное значение)
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4, 0)
print(target_train_upsampled.value_counts(normalize = 1))
print(target_train_upsampled.shape)

In [ ]:
#Протестируем функцию (верное значение)
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4, 3)
features_train_upsampled

In [ ]:
#применим функцию upsample 
#увеличим количество положительных ответов в 4 раза
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4, 1)
print(target_train_upsampled.value_counts(normalize = 1))
print(target_train_upsampled.shape)

In [ ]:
target_train_upsampled.plot(kind ='hist', bins=2, figsize=(1,1))

Мы получили сбалансированное количество классов, перейдем к обучению моеделей на новы данных.

## 3.2 Обучение моделей на сбаланированной выборке

#### Точность (score)

In [ ]:
#точность моделей на выборке с дисбалансом
all_models_accuracy(features_train, target_train, features_valid, target_valid)

In [ ]:
#точность моделей на сбалансированной выборке
all_models_accuracy(features_train_upsampled, target_train_upsampled, features_valid, target_valid)

Точность моделей изменились незначительно, но проверим изменилось ли качество ответов. 

#### Качество предсказаний

In [ ]:
#Решающее дерево
model_DTC_upsampled = DecisionTreeClassifier(random_state=123)
model_DTC_upsampled.fit(features_train_upsampled, target_train_upsampled)
DTC_prediction_upsampled = model_DTC_upsampled.predict(features_valid)
rec_prec_f1(target_valid, DTC_prediction_upsampled)

In [ ]:
#Случайный лес
model_RFC_upsampled = RandomForestClassifier(random_state=12345, n_estimators = 100)
model_RFC_upsampled.fit(features_train_upsampled, target_train_upsampled)
RFC_prediction_upsampled = model_RFC_upsampled.predict(features_valid)
rec_prec_f1(target_valid, RFC_prediction_upsampled)

In [ ]:
#Логистическая регрессия
model_LgR_upsampled = LogisticRegression(solver = 'liblinear')
model_LgR_upsampled.fit(features_train_upsampled, target_train_upsampled)
LgR_prediction_upsampled = model_LgR_upsampled.predict(features_valid)
rec_prec_f1(target_valid, LgR_prediction_upsampled)

Показаели всех моделей улучшились.   
Лучшие результаты показывает алгоритм случайный лес (RandomForestClassifier). 
На валидационной выборке RandomForestClassifier уже показывает резульаты F1 меры = 0.63, что выше целевого целевого значения.    
Попробуем улучшить модель путем изменения параметров. 

## 3.3 Улучшение модели

In [ ]:
from itertools import product
import tqdm

In [ ]:
def RandomForestQuality(features_train, target_train, features_valid, target_valid):
    
    #Параметры для перебора
    bootstrap = [True, False]
    class_weight = ['balanced', 'balanced_subsample', None]
    max_features = ['auto', 'sqrt', 'log2'] 
    max_depth = [] #диапазон изменения параметра мксимальной глубины каждого дерева
    for i in range(1, 20):
        max_depth.append(i)

    #Метод itertools.product для перебора нескольких параметров
    myproduct = product(bootstrap, class_weight, max_features, max_depth)
    
    #Строки, котоыре будут наполняться циклом при переборе параметров
    bootstrap_table = []
    class_weight_table = []
    features_table = []
    depth_table = []
    f1_table = []
    recall_table = []
    precision_table = []
    score_train_table = []
    score_valid_table = []
    
    #Цикл перебора всех параметров: bootstrap, class_weight, max_features, max_depth
    for p in tqdm.tqdm(myproduct,):
        #Обучение модели
        model_forest = RandomForestClassifier(
            bootstrap=p[0] , class_weight= p[1], max_features = p[2], max_depth = p[3], 
            n_estimators = 10, random_state=12345)
        model_forest.fit(features_train, target_train)
        prediction = model_forest.predict(features_valid) #предсказание целевого признака
        
        #расчет параметров
        f1 = f1_score(target_valid, prediction)
        recall = recall_score(target_valid, prediction)
        precision = precision_score(target_valid, prediction)
        score_train = model_forest.score(features_train, target_train)
        score_valid = model_forest.score(features_valid, target_valid)
        
        #внесение значиний параметров в строки
        bootstrap_table.append(p[0])
        class_weight_table.append(p[1])
        features_table.append(p[2])
        depth_table.append(p[3])

        #внесение значений метрик в строки
        f1_table.append(f1)
        recall_table.append(recall)
        precision_table.append(precision)
        score_train_table.append(score_train)
        score_valid_table.append(score_valid)
               
    
    #Обоъединение строк в датафрем
    quality_table = pd.DataFrame(data = (
        bootstrap_table, class_weight_table, features_table, depth_table, 
        f1_table, recall_table, precision_table, score_train_table, score_valid_table)).T
    quality_table.columns = (
        'bootstrap', 'class_weight', 'max_features', 'max_depth', 'f1', 'recall', 'precision', 'score_train', 'score_valid')
    return quality_table

    "4 параметра: features_train, target_train — признаки и целевой признак обучающей выборки"
    "features_valid, target_valid — признаки и целевой признак обучающей выборки"



In [ ]:
%%time
quality_table = RandomForestQuality(features_train_upsampled, target_train_upsampled, features_valid, target_valid)

In [ ]:
quality_table.query('score_valid>=score_train').sort_values('f1', ascending = False).head()

Лучшие параметры модели:   
model = RandomForestClassifier(bootstrap = True, class_weight = 'balanced', max_depth= 7,  n_estimators = 10, random_state=12345)

Параметр "max_features" не влияет на параметры модели, оставляем параметр без изменений (default=”auto”). 

#### Обучим финальную модель

In [ ]:
model_RFC_final = RandomForestClassifier(
    bootstrap = True, class_weight = 'balanced', max_depth= 7,  n_estimators = 100, random_state=12345)
model_RFC_final.fit(features_train_upsampled, target_train_upsampled)

In [ ]:
model_RFC_final_prediction = model_RFC_final.predict(features_valid)
rec_prec_f1(target_valid, model_RFC_final_prediction)

#### Проверим финальную модель на адекватность

In [ ]:
#Создаем константную модель
target_predict_constant = pd.Series([0]*len(target_valid))
target_predict_constant.value_counts()

In [ ]:
#Сравним показатель точности (accuracy_score) константной модели и финальной
print('accuracy_score константой модели:', accuracy_score(target_valid, target_predict_constant))
print('accuracy_score финальной модели:', accuracy_score(target_valid, model_RFC_final_prediction))
#Дополнительно сравним AUC-ROC — единственный параметр подающийся сравнению, потому что константная подель содержит только негативные ответы
print('AUC-ROC константой модели:', roc_auc_score(target_valid, target_predict_constant))
print('AUC-ROC финальной модели:', roc_auc_score(target_valid, model_RFC_final_prediction))

Финальная модель показывает результаты лучше, чем константная модель — модель можно считать адекватной. 

## Вывод по результатам исследования

1. **В первоначальные данных наблюдался значительный дисбаланс** (80% ответов целевого признака были негативными и только 20% позитивными), из-за чего обученная на этих данных модель не проходила проверку на адекватность.  **Все модели не первоначальных данных характеризовались высокой степенью ошибок и низким качеством взвешенной величины (F1)** — модели показывали низкие результаты точности и полноты. 

2. Мы устранили дисбаланс классов в обучающей выборки методом upsampling — увеличили количество значений позитивного класса в 4 раза. Так **мы достигли баланса классо в обучеющей выборки: 1 = 0.50569, 0 = 0.49431**. 

3. **На новых данных все модели показали результат выше, чем на несбалансированной выборке**. Лучшие показатели были у модели случайного леса:  
Полнота 0.5784313725490197  
Точность 0.6880466472303207  
F1-мера 0.6284953395472703  
AUC-ROC 0.755610158636319  

    Было принято решение продолжать улучшение модели на основании алгоритма случайный лес (RandomForestClassifier)

4. Улучшение модели происходило путем цикла выбора параметров, которые показывали наиболее высокое значение F1 меры и не приводили модель к переобучению. 
Изменяемые параметры: bootstrap, class_weight, max_depth от 1 до 19
Параметр n_estimators был занижен для ускорения выполнения цикла

5. **Финальные параметры выбранной модели** RandomForestClassifier(bootstrap = True, class_weight = 'balanced', max_depth= 7,  n_estimators = 100, random_state=12345).   
**Результаты модели:**  
Полнота 0.755    
Точность 0.549   
F1-мера 0.636   
AUC-ROC 0.798  
6. **Финальная модель прошла проверку на адекватность** в сравнении с контантной моделью:
accuracy_score константой модели: 0.796  
accuracy_score финальной модели: 0.8235  
AUC-ROC константой модели: 0.5  
AUC-ROC финальной модели: 0.798  

# 4. Тестирование модели

In [ ]:
model_RFC_final
model_RFC_final_prediction = model_RFC_final.predict(features_test)
rec_prec_f1(target_test, model_RFC_final_prediction)

In [ ]:
final_model_probabilities_one = model_LgR.predict_proba(features_test)[:, 1]

fpr, tpr, thresholds = roc_curve(target_test, final_model_probabilities_one) 

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

Финальная достигла заданой метрики (F1 > 0.59) и показывает адекватные результаты.  
Модель характеризуется высоким показателем полноты = 0.715 (min = 0, max = 1), поэтому она с высокой вероятностью предскажит уход клиента из банка.  
Показатель точности не высокий = 0.517 (min = 0, max = 1) — модель верно предсказывает только половину ухода клиентов.   
С точки зрения бизнеса полученная модель поможет маркетологам лучше определять килентов, которые могут уйти в ближайшее время. Важно, что модель покрывает большое количество клиентов, в данном случае это важнее чем точность.  

Для улучшения модели следует использовать данные, отражающие отношения клиента с банком в динамике. 

## Выгрузка результата

In [ ]:
results = pd.DataFrame(model_RFC_final_prediction)
results.head(5)

In [ ]:
results.to_csv('/kaggle/working/Churn_Modelling_results.csv')